# SSH Signed Request Auth

This notebook demonstrates how to make authenticated API requests using SSH signing. It provides examples for uploading campaign measurements and performing real-time analysis by making signed HTTP requests to a REST API endpoint.


In [1]:
from rfscope.crypto.api_requests import signed_request
from datetime import datetime
import json

API_URL = "http://127.0.0.1:8000/api"

def request(endpoint, payload={}, method="POST"):
    response = signed_request(
    f"{API_URL}/{endpoint}",
    json_body=payload,
    priv_path="~/.ssh/id_ed25519",
    verify_tls=True,
    method=method
    )

    print("Status:", response.status_code)
    print("Reason:", response.reason)
    try: print("JSON:", json.dumps(response.json(), indent=4))
    except: pass


## Campaign measure upload

In this section, we demonstrate how to upload campaign measurements by making a signed request to the monitoring/measures/ endpoint. The payload includes campaign ID, timestamp, PSD data, and GPS coordinates.


In [7]:
payload = {
    "campaign": 119,
    "timestamp": datetime.now().timestamp(),
    "psd": "XXXXXX",
    "gps_lat": 0.0,
    "gps_lng": 0.0,
}

In [8]:
request('monitoring/measures/', payload)

Status: 201
Reason: Created
JSON: {
    "campaign": 119,
    "psd": "XXXXXX",
    "timestamp": "2025-10-28T08:32:21.276779-05:00",
    "gps_lat": 0.0,
    "gps_lng": 0.0
}


In [9]:
request('monitoring/measures/', {})


Status: 400
Reason: Bad Request
JSON: {
    "campaign": [
        "This field is required."
    ],
    "psd": [
        "This field is required."
    ],
    "timestamp": [
        "This field is required."
    ],
    "gps_lat": [
        "This field is required."
    ],
    "gps_lng": [
        "This field is required."
    ]
}


## Real-time analysis

This section demonstrates how to perform real-time spectrum analysis using the monitoring/temporal-measures/ endpoint. The payload includes timestamp, PSD data, frequency parameters (start, end, resolution), receiver port number, and GPS coordinates.


In [27]:
payload = {
    "timestamp": datetime.now().timestamp(),
    "psd": "XXXXXX",
    "port": 2,
    "gps_lat": 1.2,
    "gps_lng": 3.4,
}

In [28]:
request('monitoring/temporal-measures/', payload)

Status: 201
Reason: Created
JSON: {
    "psd": "XXXXXX",
    "port": 2,
    "timestamp": "2025-10-28T08:48:26.521708-05:00",
    "gps_lat": 1.2,
    "gps_lng": 3.4
}


In [29]:
request('monitoring/temporal-measures/', {})

Status: 400
Reason: Bad Request
JSON: {
    "psd": [
        "This field is required."
    ],
    "port": [
        "This field is required."
    ],
    "timestamp": [
        "This field is required."
    ],
    "gps_lat": [
        "This field is required."
    ],
    "gps_lng": [
        "This field is required."
    ]
}


## Query campaigns and real-time analysis parameters

This section demonstrates how to query information about active campaigns and real-time analysis tasks by making GET requests to the monitoring/device-tasks/ endpoint. This allows retrieving the current configuration and status of monitoring activities.


In [157]:
request('monitoring/device-tasks/', method="GET")

Status: 200
Reason: OK
JSON: {
    "campaigns": [
        {
            "campaign_id": 118,
            "status": "scheduled",
            "start_freq_hz": 439000000,
            "end_freq_hz": 1109000000,
            "resolution_hz": 300000.0,
            "acquisition_period_s": 43200,
            "timeframe": {
                "start": 1761353433000,
                "end": 1768294233000
            },
            "antenna_port": 2
        }
    ],
    "real-time": {
        "active": true,
        "start_freq_hz": 69,
        "end_freq_hz": 169,
        "resolution_hz": 1000.0,
        "antenna_gain_dbi": 19.2,
        "antenna_impedance_ohm": 75.0,
        "demodulation": {
            "type": "",
            "bandwidth_hz": "",
            "center_freq_hz": ""
        }
    }
}
